# Day 4: Passport Processing

[*Advent of Code 2020 day 4*](https://adventofcode.com/2020/day/4) and [*solution megathread*](https://redd.it/k6e8sw)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/04/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F04%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
testdata = '''ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in'''.splitlines()

inputdata = downloaded['input'].splitlines()

In [6]:
import functools

requiredFields = {'byr': 'Birth Year',
                  'iyr': 'Issue Year',
                  'eyr': 'Expiration Year',
                  'hgt': 'Height',
                  'hcl': 'Hair Color',
                  'ecl': 'Eye Color',
                  'pid': 'Passport ID'}
# 'cid': 'Country ID' is not actually a required field


def parseItems(data):
    items, i = ([dict()], 0)
    for line in data:
        if len(line) == 0:
            i += 1
            items.append(dict())
        else:
            for token in line.split():
                key, value = token.split(':', 1)
                items[i][key] = value
    return items

In [7]:
passports = parseItems(testdata)
print(passports)

for passport in passports:
    validity = functools.reduce(lambda a, b: a & b,
                                [field in passport for
                                 field in requiredFields.keys()])
    print(f"{passport} is a {validity} item")

[{'ecl': 'gry', 'pid': '860033327', 'eyr': '2020', 'hcl': '#fffffd', 'byr': '1937', 'iyr': '2017', 'cid': '147', 'hgt': '183cm'}, {'iyr': '2013', 'ecl': 'amb', 'cid': '350', 'eyr': '2023', 'pid': '028048884', 'hcl': '#cfa07d', 'byr': '1929'}, {'hcl': '#ae17e1', 'iyr': '2013', 'eyr': '2024', 'ecl': 'brn', 'pid': '760753108', 'byr': '1931', 'hgt': '179cm'}, {'hcl': '#cfa07d', 'eyr': '2025', 'pid': '166559648', 'iyr': '2011', 'ecl': 'brn', 'hgt': '59in'}]
{'ecl': 'gry', 'pid': '860033327', 'eyr': '2020', 'hcl': '#fffffd', 'byr': '1937', 'iyr': '2017', 'cid': '147', 'hgt': '183cm'} is a True item
{'iyr': '2013', 'ecl': 'amb', 'cid': '350', 'eyr': '2023', 'pid': '028048884', 'hcl': '#cfa07d', 'byr': '1929'} is a False item
{'hcl': '#ae17e1', 'iyr': '2013', 'eyr': '2024', 'ecl': 'brn', 'pid': '760753108', 'byr': '1931', 'hgt': '179cm'} is a True item
{'hcl': '#cfa07d', 'eyr': '2025', 'pid': '166559648', 'iyr': '2011', 'ecl': 'brn', 'hgt': '59in'} is a False item


In [8]:
passports = parseItems(inputdata)
answer = sum([functools.reduce(lambda a, b: a & b,
                               [field in passport for
                                field in requiredFields.keys()])
              for passport in passports])
print(answer)

206


In [9]:
HTML(downloaded['part1_footer'])

## Part Two

In [10]:
HTML(downloaded['part2'])

In [11]:
def validate(key, value):
    def switch(option, cases, default):
        if option in cases:
            return cases[option]
        else:
            return default

    cases = {
        'byr': lambda byr:
            len(byr) == 4 and 1920 <= int(byr) and int(byr) <= 2002,
        'iyr': lambda iyr:
            len(iyr) == 4 and 2010 <= int(iyr) and int(iyr) <= 2020,
        'eyr': lambda eyr:
            len(eyr) == 4 and 2020 <= int(eyr) and int(eyr) <= 2030,
        'hgt': lambda hgt:
            (hgt[-2:] == 'cm'
             and 150 <= int(hgt[:-2])
             and int(hgt[:-2]) <= 193)
            or (hgt[-2:] == 'in'
                and 59 <= int(hgt[:-2])
                and int(hgt[:-2]) <= 76),
        'hcl': lambda hcl:
            len(hcl) == 7
            and hcl[0] == '#'
            and functools.reduce(lambda a, b: a & b,
                                 [c in '0123456789abcdef'
                                  for c in hcl[1:]]),
        'ecl': lambda ecl:
            ecl in {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'},
        'pid': lambda pid:
            len(pid) == 9
            and functools.reduce(lambda a, b: a & b,
                                 [c in '0123456789'
                                  for c in pid[1:]]),
        'cid': lambda ignored: True
    }

    default = lambda ignored: False

    return(switch(key, cases, default)(value))

38:5: E731 do not assign a lambda expression, use a def


In [12]:
print("Tests passing:",
      functools.reduce(lambda a, b: a & b,
                       [not validate('iyr', '2009'),
                        validate('iyr', '2012'),
                        not validate('iyr', '2021'),
                        validate('byr', '1930'),
                        not validate('acv', '2021'),
                        validate('hgt', '180cm'),
                        not validate('hgt', '210cm'),
                        not validate('hgt', '210sd'),
                        not validate('hgt', '12in'),
                        validate('hgt', '60in'),
                        validate('hcl', '#ffffff'),
                        not validate('hcl', '#fffffff'),
                        validate('ecl', 'blu'),
                        not validate('ecl', 'asd'),
                        validate('pid', '123456789'),
                        not validate('pid', '1234567a9')]))

Tests passing: True


In [13]:

answer = sum([
    functools.reduce(lambda a, b: a & b,
                     [field in passport
                      and validate(field, passport[field])
                      for field in requiredFields.keys()])
    for passport in passports])
print(answer)

123


In [14]:
HTML(downloaded['part2_footer'])